In [ ]:
from os import environ
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory,
    ConversationTokenBufferMemory,
)
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent, AgentType, load_tools

In [2]:
# Read API KEY
load_dotenv()
TOKEN_OPENAI = environ['TOKEN_OPENAI']

In [3]:
# Load OpenAI LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=TOKEN_OPENAI)

# Regular agent

In [22]:
# Create tools
@tool
def investing(capital: int, tenure: float, interest: float) -> int:
    """Calculate the final value of investment given the capital, interest rate per annum, and tenure"""
    invest = capital * (1 + interest)**tenure
    return invest

@tool
def calculate_dy(current_price: int, dividend: int) -> float:
    """Calculate dividend yield based on the dividend and and current price"""
    dividend_yield = round(dividend/current_price, 3)
    return dividend_yield

@tool
def entry_criteria(dividend_yield: float, current_price: int, yearly_payout: bool) -> str:
    """Decide whether to buy the stock (entry) based on the dividend yield, current price, and yearly payout"""
    if yearly_payout == True:
        if dividend_yield >= 0.04 or current_price <= 1000:
            decision = "buy the stock"
        else:
            decision = "do not buy the stock"
    else:
        if dividend_yield >= 0.6 and current_price <= 3000:
            decision = "buy the stock"
        else:
            decision = "do not buy the stock"
    
    return decision

@tool
def is_expensive(price: int) -> bool:
    """Check if the price is expensive"""
    result = True if price > 2000 else False
    return result


In [87]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer not more than 20 words"),
        MessagesPlaceholder("history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [7]:
tools = [investing, calculate_dy, entry_criteria]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
response = agent_executor.invoke({"input": "Hey, what's up?"})
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Hello! How can I assist you today?

> Finished chain.


{'input': "Hey, what's up?", 'output': 'Hello! How can I assist you today?'}

In [13]:
response = agent_executor.invoke({"input": "Define finance in less than 10 words."})
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Management of money, investments, and financial resources.

> Finished chain.


{'input': 'Define finance in less than 10 words.',
 'output': 'Management of money, investments, and financial resources.'}

In [ ]:
response = agent_executor.invoke(
    {"input": "if I buy a stock with the dividend of 30 when the price is 270, how much is the dividend yield?"}
)
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `calculate_dy` with `{'current_price': 270, 'dividend': 30}`


0.111The dividend yield is 11.1%.

> Finished chain.


{'input': 'if I buy a stock with the dividend of 30 when the price is 270, how much is the dividend yield?',
 'output': 'The dividend yield is 11.1%.'}

In [ ]:
response = agent_executor.invoke(
    {"input": "if I buy a stock with the dividend of 30 when the price is 270. The stock pays dividend every year. Should I buy the stock?"}
)
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `calculate_dy` with `{'current_price': 270, 'dividend': 30}`


0.111
Invoking: `entry_criteria` with `{'dividend_yield': 0.111, 'current_price': 270, 'yearly_payout': True}`


buy the stockYes, you should buy the stock.

> Finished chain.


{'input': 'if I buy a stock with the dividend of 30 when the price is 270. The stock pays dividend every year. Should I buy the stock?',
 'output': 'Yes, you should buy the stock.'}

In [60]:
response = agent_executor.invoke({"input": "The stock pays dividend every year. Should I buy the stock?"})
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I need the dividend yield, current price, and yearly payout details to decide.

> Finished chain.


{'input': 'The stock pays dividend every year. Should I buy the stock?',
 'output': 'I need the dividend yield, current price, and yearly payout details to decide.'}

# Add memory

In [95]:
# Create memory
memory = ConversationBufferMemory(llm=llm, memory_key="history")
# Prepopulate the memory with chat history
history_input = ["if I buy a stock with the dividend of 30 when the price is 270, how much is the dividend yield?"]
history_output = ["The dividend yield is 11.1%."]
if history_input is not None:
    for inp, outp in zip(history_input, history_output):
        memory.save_context({"input": inp}, {"output": outp})

# Create agent
tools = [investing, calculate_dy, entry_criteria]
agent_memory = create_tool_calling_agent(llm, tools, prompt)
agent_memory = AgentExecutor(
    agent=agent_memory, tools=tools, verbose=True
)

In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [93]:
response = agent_memory.invoke({"input": "The stock pays dividend every year. Should I buy the stock?"})
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


ValueError: variable history should be a list of base messages, got Human: if I buy a stock with the dividend of 30 when the price is 270, how much is the dividend yield?
AI: The dividend yield is 11.1%. of type <class 'str'>

# Zero-shot agent
## External tools

In [37]:
# Zero-shot agent with external tools
tools = load_tools(["llm-math"], llm=llm)
agent_external = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [39]:
response = agent_external.run("what is 6 * 11?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I will calculate the product of 6 and 11 using the calculator tool.

Action: Calculator
Action Input: 6 * 11
Observation: Answer: 66
Thought:I now know the final answer. 

Final Answer: 6 * 11 is 66.

> Finished chain.


In [38]:
response = agent_external.run("(4 + 5)/3*2 is ")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


To solve the expression \((4 + 5)/3*2\), I need to follow the order of operations: parentheses, division, and then multiplication.

1. Calculate the expression inside the parentheses: \(4 + 5\).
2. Divide the result by 3.
3. Multiply the result by 2.

Action: Calculator
Action Input: (4 + 5)/3*2
Observation: Answer: 6.0
Thought:I now know the final answer. 

Final Answer: The result of the expression \((4 + 5)/3*2\) is 6.0.

> Finished chain.


## Custom tools

In [ ]:
# Create zero-shot agent
agent_zeroshot = initialize_agent(
    tools=[investing, calculate_dy, entry_criteria],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [32]:
response = agent_zeroshot.run("if I buy a stock with the dividend of 30 when the price is 270, how much is the dividend yield?")
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: To calculate the dividend yield, I need to use the dividend and the current price of the stock.
Action:
```
{
  "action": "calculate_dy",
  "action_input": {
    "current_price": 270,
    "dividend": 30
  }
}
```
Observation: 0.111
Thought:The dividend yield for a stock with a dividend of 30 and a current price of 270 is 0.111, or 11.1%. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "The dividend yield is 11.1%."
}
```

> Finished chain.


'The dividend yield is 11.1%.'

In [31]:
response = agent_zeroshot.run("if I buy a stock with the dividend of 30 when the price is 270. The stock pays dividend every year. Should I buy the stock?")
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: To determine if you should buy the stock, I need to calculate the dividend yield and then use the entry criteria to make a decision.
Action:
```
{
  "action": "calculate_dy",
  "action_input": {
    "current_price": 270,
    "dividend": 30
  }
}
```
Observation: 0.111
Thought:To determine if you should buy the stock, I need to use the entry criteria based on the calculated dividend yield, current price, and the fact that the stock pays dividends yearly.

Action:
```
{
  "action": "entry_criteria",
  "action_input": {
    "dividend_yield": 0.111,
    "current_price": 270,
    "yearly_payout": true
  }
}
```

Observation: buy the stock
Thought:I know what to respond. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "Based on the calculated dividend yield of 11.1% and the fact that the stock pays dividends yearly, it is recommended to buy the stock."
}
```

> Finished chain.


'Based on the calculated dividend yield of 11.1% and the fact that the stock pays dividends yearly, it is recommended to buy the stock.'

## Use chat prompt template

In [ ]:
# BELUM BERHASIL
# Create zero-shot agent
agent_zeroshot = initialize_agent(
    tools=[investing, calculate_dy, entry_criteria],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"prompt": prompt}
)

In [35]:
response = agent_zeroshot.run("if I buy a stock with the dividend of 30 when the price is 270. The stock pays dividend every year. Should I buy the stock?")
response

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: To determine whether you should buy the stock, I need to calculate the dividend yield and then use the entry criteria to make a decision. First, I'll calculate the dividend yield.
Action:
```
{
  "action": "calculate_dy",
  "action_input": {
    "current_price": 270,
    "dividend": 30
  }
}
```
Observation: 0.111
Thought:To determine whether you should buy the stock, I need to use the entry criteria based on the calculated dividend yield, current price, and the fact that the stock pays dividends yearly. Let's proceed with that.

Action:
```
{
  "action": "entry_criteria",
  "action_input": {
    "dividend_yield": 0.111,
    "current_price": 270,
    "yearly_payout": true
  }
}
```

Observation: buy the stock
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "Based on the calculated dividend yield of 11.1% and the entry criteria, it is recommended to buy the stock."
}
```

> Finished chain.


'Based on the calculated dividend yield of 11.1% and the entry criteria, it is recommended to buy the stock.'

## Add memory

In [ ]:
# Initialize memory
agent_zeroshot_memory = ConversationBufferMemory(llm=agent_zeroshot, memory_key="history")

# Prepopulate the memory with chat history
history_input = ["if I buy a stock with the dividend of 30 when the price is 270, how much is the dividend yield?"]
history_output = ["The dividend yield is 11.1%."]
if history_input is not None:
    for inp, outp in zip(history_input, history_output):
        agent_zeroshot_memory.save_context({"input": inp}, {"output": outp})

In [ ]:
agent_zeroshot_memory = ConversationBufferMemory(llm=agent_zeroshot, memory_key="history")
agent_zeroshot_memory

TypeError: 'ConversationBufferMemory' object is not callable

In [82]:
response = agent_zeroshot_memory(
    "if I buy a stock with the dividend of 30 when the price is 270. The stock pays dividend every year. Should I buy the stock?"
)
response

TypeError: 'ConversationBufferMemory' object is not callable